# Factor Investing and Fama-French model

**Copyright &copy; 2020 Ondrej Martinsky, All rights reserved**

[www.github.com/omartinsky/FamaFrench](https://github.com/omartinsky/FamaFrench)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wx
from ipywidgets import interact
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from pandas.plotting import register_matplotlib_converters
from IPython.core.display import display, HTML
import famafrench
register_matplotlib_converters()

/var/folders/fj/4vv2tyt94_l44bxlschbrx5h0000gn/T/ipykernel_25238/3355595522.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
/Users/marmarti/Documents/FinanceProjects/mariomff1/MMFF20240811/famafrench.py:43: FutureWarning: DataFrame.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  p = p.replace(-99.99, math.nan).replace(999) / 100
/Users/marmarti/Documents/FinanceProjects/mariomff1/MMFF20240811/famafrench.py:43: FutureWarning: DataFrame.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  p = p.replace(-99.99, math.nan).replace(999) / 100
/Users/marmarti/Documents/FinanceProjects/mariomff1/MMFF20240811/famafrench.py:43: FutureWarning: DataFrame.

In [2]:
def create_ma_radios(value):
    return wx.RadioButtons(value=value, options=famafrench.lookup_ma.keys(), description='Roll Avg')

# Fama French

$ r = R_f + b_m \cdot \text{MER} + b_s \cdot \text{SMB} + b_v \cdot \text{HML} + b_p \cdot \text{RMW} + b_i \cdot \text{CMA} $

$r$ is expected portfolio return \
$b_m$, $b_s$, $b_v$, $b_p$, $b_i$ are factor sensitivities

# Factors Analysis
$\text{MER}$ is excess return of a broad market over the risk-free rate ($R_m - R_f$) (**market factor**) \
$\text{SMB}$ is the average return on the *small* stock portfolios minus *big* stock portfolios (**size factor**) \
$\text{HML}$ is the average return on *high* book value portfolios minus *low* book value (**value factor**)  \
$\text{RMW}$ is the average return on *robust* minus *weak* operating profitability portfolios (**profitability factor**) \
$\text{CMA}$ Conservative minus aggresive investments, proxied by the y/y growth in company asset value (**investment factor**)

Definitions: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_5_factors_2x3.html

In [3]:
display(HTML('<b>Factors time series</b>'), famafrench.df_factors)
display(HTML('<b>Factors Statistics</b>'), famafrench.df_factors.describe().T)
display(HTML('<b>Factors Correlation Matrix</b>'), famafrench.df_factors.corr())

,MER,SMB,HML,RMW,CMA,RF
1963-07-01,-0.0067,0.0002,-0.0035,0.0003,0.0013,0.00012
1963-07-02,0.0079,-0.0028,0.0028,-0.0008,-0.0021,0.00012
1963-07-03,0.0063,-0.0018,-0.0010,0.0013,-0.0025,0.00012
1963-07-05,0.0040,0.0009,-0.0028,0.0007,-0.0030,0.00012
1963-07-08,-0.0063,0.0007,-0.0020,-0.0027,0.0006,0.00012
...,...,...,...,...,...,...
2024-06-24,-0.0026,0.0063,0.0111,0.0000,0.0065,0.00022
2024-06-25,0.0031,-0.0081,-0.0120,0.0010,-0.0121,0.00022
2024-06-26,0.0016,0.0000,-0.0020,0.0045,-0.0027,0.00022
2024-06-27,0.0014,0.0051,-0.0036,-0.0101,-0.0012,0.00022


,count,mean,std,min,25%,50%,75%,max
MER,15353.0,0.000279,0.010202,-0.1744,-0.00420,0.00050,0.00510,0.11350
SMB,15353.0,0.000061,0.005470,-0.1119,-0.00280,0.00020,0.00300,0.06170
HML,15353.0,0.000139,0.005835,-0.0502,-0.00240,0.00010,0.00250,0.06730
RMW,15353.0,0.000140,0.004009,-0.0301,-0.00170,0.00010,0.00190,0.04510
CMA,15353.0,0.000122,0.003794,-0.0587,-0.00180,0.00010,0.00200,0.02530
RF,15353.0,0.000173,0.000125,0.0000,0.00007,0.00018,0.00024,0.00061


,MER,SMB,HML,RMW,CMA,RF
MER,1.000000,-0.061013,-0.160724,-0.208133,-0.356788,-0.019111
SMB,-0.061013,1.000000,0.136229,-0.278008,0.030084,-0.012744
HML,-0.160724,0.136229,1.000000,0.058633,0.560954,0.016867
RMW,-0.208133,-0.278008,0.058633,1.000000,0.126404,-0.001256
CMA,-0.356788,0.030084,0.560954,0.126404,1.000000,0.015742
RF,-0.019111,-0.012744,0.016867,-0.001256,0.015742,1.000000


In [4]:
display(HTML("<b>Distribution Analysis</b>"))
    
wx_ff=wx.Dropdown(options=famafrench.lookup_factors.keys(), description='Factor')
wx_ma=create_ma_radios('1D')
wx_out1=wx.interactive_output(famafrench.plot_factor_hists, {'ff':wx_ff, 'ma':wx_ma})
wx_out2=wx.interactive_output(famafrench.plot_factor_timeseries, {'ff':wx_ff, 'ma':wx_ma})
display(wx.HBox([wx_ff, wx_ma]), wx_out1, wx_out2)

Output()

Output()

# Portfolio Analysis

In [5]:
def create_factor_radios():
    return {
        'f_mer': wx.Checkbox(description='MER', indent=False, layout=chkbox_layout, value=True),
        'f_smb': wx.Checkbox(description='SMB', indent=False, layout=chkbox_layout, value=True),
        'f_hml': wx.Checkbox(description='HML', indent=False, layout=chkbox_layout, value=True),
        'f_rmw': wx.Checkbox(description='RMW', indent=False, layout=chkbox_layout),
        'f_cma': wx.Checkbox(description='CMA', indent=False, layout=chkbox_layout),
    }

def create_portfolio_dropdown():
    return wx.Dropdown(options=famafrench.portfolios.keys(), description='Portfolio', layout=wx.Layout(width='500px'))


chkbox_layout = wx.Layout(width='70px')
wx_pname = create_portfolio_dropdown()
wx_ma = create_ma_radios('3Y')
wx_factor_list = create_factor_radios()
wx_daterange=wx.SelectionRangeSlider(options=range(1960,2025), index=(0, len(range(1960,2025))-1), description='Range', continuous_update=False, layout=wx.Layout(width='500px'))
wx_out=wx.interactive_output(famafrench.fit_portfolio_returns, {'pname':wx_pname, **wx_factor_list , 'ma': wx_ma, 'daterange': wx_daterange})
wx_out.layout.height='700px'
wx_out_r2hist = wx.interactive_output(famafrench.draw_R2_hist, {'pname':wx_pname, 'daterange': wx_daterange})
wx_out_r2series = wx.interactive_output(famafrench.draw_R2_series, {'pname':wx_pname, **wx_factor_list, 'daterange': wx_daterange})
display(
    wx_pname, 
    wx.HBox([wx_daterange, *(wx_factor_list.values())]),
    wx.HBox([wx_out_r2hist, wx_out_r2series]),
    wx.HBox([wx_out, wx_ma]))

Dropdown(description='Portfolio', layout=Layout(width='500px'), options=('1. Univariate SMB ➤   Value Weighted…

In [6]:
def create_factor_radios_2():
    return {
        'f_mer': wx.Checkbox(description='MER', indent=False, layout=chkbox_layout, value=True),
        'f_smb': wx.Checkbox(description='SMB', indent=False, layout=chkbox_layout, value=True),
        'f_hml': wx.Checkbox(description='HML', indent=False, layout=chkbox_layout, value=True),
        'f_rmw': wx.Checkbox(description='RMW', indent=False, layout=chkbox_layout),
        'f_cma': wx.Checkbox(description='CMA', indent=False, layout=chkbox_layout),
    }

def create_portfolio_dropdown():
    return wx.Dropdown(options=famafrench.portfolios.keys(), description='Portfolio', layout=wx.Layout(width='500px'))


chkbox_layout = wx.Layout(width='70px')
wx_pname = create_portfolio_dropdown()
wx_ma = create_ma_radios('3Y')
wx_factor_list = create_factor_radios()
wx_daterange=wx.SelectionRangeSlider(options=range(1960,2021), index=(0, len(range(1960,2021))-1), description='Range', continuous_update=False, layout=wx.Layout(width='500px'))
wx_out=wx.interactive_output(famafrench.fit_portfolio_returns, {'pname':wx_pname, **wx_factor_list , 'ma': wx_ma, 'daterange': wx_daterange})
wx_out.layout.height='700px'
wx_out_r2hist = wx.interactive_output(famafrench.draw_R2_hist, {'pname':wx_pname, 'daterange': wx_daterange})
wx_out_r2series = wx.interactive_output(famafrench.draw_R2_series, {'pname':wx_pname, **wx_factor_list, 'daterange': wx_daterange})
display(
    wx_pname, 
    wx.HBox([wx_daterange, *(wx_factor_list.values())]),
    wx.HBox([wx_out_r2hist, wx_out_r2series]),
    wx.HBox([wx_out, wx_ma]))

Dropdown(description='Portfolio', layout=Layout(width='500px'), options=('1. Univariate SMB ➤   Value Weighted…

In [7]:
def create_factor_radios_3():
    return {
        'f_mer': wx.Checkbox(description='MER', indent=False, layout=chkbox_layout, value=True),
        'f_smb': wx.Checkbox(description='SMB', indent=False, layout=chkbox_layout, value=True),
        'f_hml': wx.Checkbox(description='HML', indent=False, layout=chkbox_layout, value=True),
        'f_rmw': wx.Checkbox(description='RMW', indent=False, layout=chkbox_layout),
        'f_cma': wx.Checkbox(description='CMA', indent=False, layout=chkbox_layout),
    }

def create_portfolio_dropdown():
    return wx.Dropdown(options=famafrench.portfolios.keys(), description='Portfolio', layout=wx.Layout(width='500px'))


chkbox_layout = wx.Layout(width='70px')
wx_pname = create_portfolio_dropdown()
wx_ma = create_ma_radios('3Y')
wx_factor_list = create_factor_radios()
wx_daterange=wx.SelectionRangeSlider(options=range(1960,2021), index=(0, len(range(1960,2021))-1), description='Range', continuous_update=False, layout=wx.Layout(width='500px'))
wx_out=wx.interactive_output(famafrench.fit_portfolio_returns, {'pname':wx_pname, **wx_factor_list , 'ma': wx_ma, 'daterange': wx_daterange})
wx_out.layout.height='700px'
wx_out_r2hist = wx.interactive_output(famafrench.draw_R2_hist, {'pname':wx_pname, 'daterange': wx_daterange})
wx_out_r2series = wx.interactive_output(famafrench.draw_R2_series, {'pname':wx_pname, **wx_factor_list, 'daterange': wx_daterange})
display(
    wx_pname, 
    wx.HBox([wx_daterange, *(wx_factor_list.values())]),
    wx.HBox([wx_out_r2hist, wx_out_r2series]),
    wx.HBox([wx_out, wx_ma]))

Dropdown(description='Portfolio', layout=Layout(width='500px'), options=('1. Univariate SMB ➤   Value Weighted…